### PromptTemplate
- PromptTemplate : 사용자의 입력을 가공
- ChatPromptTemplate : 시스템 메시지를 설정하여 LLM에 역할 부여

#### 01 환경세팅

In [1]:
!pip install --quiet langchain openai langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 1.6 MB/s eta 0:00:00


In [2]:
# 사용자 환경 설정
# modulabs통해 발급받은 API-KEY 사용
# 추후 open-ai, gemini KEY 발급시에도 아래처럼 환경 설정 가능

# 01 Colab용 비밀 키 불러오기
from google.colab import userdata
import os
from openai import OpenAI  # openai-python SDK

# 02 OpenAI API 키 설정
api_key = userdata.get("modulabsKey")  # Colab 사용자 환경변수 탭에서 등록한 키
os.environ["OPENAI_API_KEY"] = api_key
client = OpenAI(api_key=api_key)

# 03 버전 확인 (선택)
import openai
import langchain
print("LangChain ver:", langchain.__version__)
print("OpenAI SDK ver:", openai.__version__)

LangChain ver: 0.3.25
OpenAI SDK ver: 1.86.0


#### 02 PromptTemplate

In [3]:
from langchain.prompts import PromptTemplate

prompt = (
    PromptTemplate.from_template(
    """
    너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 {개수}추천하고,
    그 요리의 레시피를 제시해줘. 내가 가진 재료는 아래와 같아.
    <재료>
    {재료}
    """
    )
)

prompt

PromptTemplate(input_variables=['개수', '재료'], input_types={}, partial_variables={}, template='\n    너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 {개수}추천하고,\n    그 요리의 레시피를 제시해줘. 내가 가진 재료는 아래와 같아.\n    <재료>\n    {재료}\n    ')

In [4]:
prompt.format(개수=3, 재료="사과,양파,계란")

'\n    너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 3추천하고,\n    그 요리의 레시피를 제시해줘. 내가 가진 재료는 아래와 같아.\n    <재료>\n    사과,양파,계란\n    '

##### 교재와 다르게 코드 작성

In [19]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
import os

# API 키 설정 (Colab에서는 userdata.get()으로 해도 됨)
# os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

# 프롬프트 템플릿 구성: 메시지 리스트를 ChatPromptTemplate에 넣는다!
chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="너는 항상 밝은 말투로 대화하는 챗봇이야. 답변의 끝에 이모티콘을 붙여줘."),
    HumanMessage(content="오늘은 날씨가 어때?"),
    AIMessage(content="오늘은 날씨가 아주 좋아요! ☀️"),
    HumanMessage(content="{input}")
])

# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini")

# LLMChain 구성
chain = LLMChain(llm=model, prompt=chat_prompt)

# 질문 전달 (딕셔너리로 넘겨야 함!)
response = chain.invoke({"input": "오늘 너의 기분은 어때?"})
print(response["text"])

무엇을 도와드릴까요? 궁금한 것이 있으면 언제든지 말씀해 주세요! 😊


#### 03 Few-shot 예제

In [20]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
examples = [
    {
        "question": "아이유로 삼행시 만들어줘",
        "answer":
                """
                아: 아이유는
                이: 이런 강의를 들을 이
                유: 유가 없다.
                """
    }
]

example_prompt = PromptTemplate(
                                input_variables=["question", "answer"],
                                template="Question: {question}\n{answer}"
                                )

print(example_prompt.format(**examples[0]))

Question: 아이유로 삼행시 만들어줘

                아: 아이유는
                이: 이런 강의를 들을 이
                유: 유가 없다.
                


In [21]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)

print(prompt.format(input="호날두로 삼행시 만들어줘"))

Question: 아이유로 삼행시 만들어줘

                아: 아이유는
                이: 이런 강의를 들을 이
                유: 유가 없다.
                

Question: 호날두로 삼행시 만들어줘


##### Fewshot미적용
- 교재에서는 few-shot없으면 엉뚱한 답을 내놓지만, 현재는 모델이 개선되어서   
  삼행시를 제대로 내놓는 것을 확인할 수 있음

In [22]:
model = ChatOpenAI(model_name = "gpt-4o-mini",temperature=1)
result = model.invoke("호날두로 삼행시 만들어줘")
print(result.content)

호: 호기롭게 필드를 누비고  
날: 날카로운 슈팅으로 골을 노려  
두: 두려움 없이 승리를 향해 나아가네!  


##### Fewshot적용

In [23]:
result = model.invoke(prompt.format(input="호날두로 삼행시 만들어줘"))
print(result.content)

호: 호기심 가득한 그라운드에서  
날: 날카로운 슈팅으로 적을 무너뜨리고  
두: 두려움 없이 꿈을 향해 나아간다.


#### 04 Partial프롬프트 템플릿
- 프롬프트템플릿(PromptTemplate)에 여러 매개변수 입력시, 템플릿에 매개변수를   
  한꺼번에 입력해야 하는 불편함이 존재함
- 템플릿에 들어가는 매개변수 따로따로 입력받고 싶다면 partial()활용

In [24]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("나이: {age} \n 직업: {job}")
partial_prompt = prompt.partial(age="20")
print(partial_prompt.format(job="개발자"))

나이: 20 
 직업: 개발자


In [25]:
from datetime import datetime

def _get_datetime():
  now = datetime.now()
  return now.strftime("%m/%d/%Y, %H:%M:%S")

In [26]:
prompt = PromptTemplate(
    template = "Tell me a {adjective} joke about the day {date}",
    input_variables = ["adjective","date"],
)
partial_prompt = prompt.partial(date=_get_datetime)
print(partial_prompt.format(adjective="funny"))

Tell me a funny joke about the day 06/23/2025, 07:57:00
